### Introduction

The goal of this notebook is to introduce TS-tools, a useful open-source software package that facilitate the automated generation of reaction profiles, starting from a simple reaction SMILES input (see figure below for a schematic overview of the algorithm; more information about the technical detail can be found in the associated paper -- https://onlinelibrary.wiley.com/doi/10.1002/jcc.27374). Compared to autodE, TS-tools is particularly suited for transition states involving 3 or more reactants, which are relevant for example in solvent- and autocatalysis, as well as enzymatic reactivity. Additionally, it is exceptionally fast, generating xTB quality TS in less than 1 CPU hour typically, making it more suitable for high-throughput screening campaigns.

<div style="text-align: center;">
<img src="data/TSTools.png" alt="alt text" width="600"/>
</div>

### TS-tools

Please start by installing -- and activating -- the conda environment present in the repository (conda env create -f environment.yml)

Next, we need to install TS-tools. This can be done by accessing the package through the command line (cd TS-tools-master), followed by a local installation (pip install .)

In [ ]:
import os

home_dir = os.getcwd()

# Change to the directory containing the TS-tools package
os.chdir('TS-tools-master')

# Run the pip install command
!pip install .

os.chdir(home_dir)

A detailed explanation of how the TS-tools code can be executed can be found in the readme.md file present in the repository. To execute its full workflow, Gaussian16 needs to be available. Since Gaussian16 is not installed on your local computers, we will focus here exclusively on the generation of preliminary TS guesses from (xTB-generated) reactive paths.

### Generate TS guesses

The reaction SMILES we will try to generate TS guesses for can be found in 'TS-tools-master/data/test_aldol.txt'. They are schematically represented in the figure below.

<div style="text-align: center;">
<img src="data/aldol.png" width="500" >
</div>

In [ ]:
from tstools.ts_optimizer import TSOptimizer
import multiprocessing
import concurrent.futures
from tstools.utils import remove_files_in_directory, get_reaction_list

reaction_list = get_reaction_list('TS-tools-master/data/test_aldol.txt')

In [ ]:
# define some auxiliary functions to interact with the autodE code; these are adapted from the run_scripts available in the 'TS-tools-master' folder.
def obtain_individual_guesses(ts_optimizer):
    """
    Optimize an individual transition state.

    Parameters:
    - ts_optimizer: Instance of TSOptimizer.

    Returns:
    - int or None: Reaction ID if a transition state is found, None otherwise.
    """
    reactive_complex_factor_value = 1.5
    for _ in range(3):
        try:
            ts_optimizer.set_ts_guess_list(reactive_complex_factor_value)
            remove_files_in_directory(os.getcwd())
            if len(ts_optimizer.ts_guess_list) is not None:
                return ts_optimizer.rxn_id
        except Exception as e:
            print(e)
            continue

def obtain_ts_guesses(target_dir, reaction_list, xtb_external_path, solvent,
                             reactive_complex_factor_list_intermolecular,
                             reactive_complex_factor_list_intramolecular, freq_cut_off):
    """
    Obtain transition states for a list of reactions.

    Parameters:
    - target_dir (str): Target directory.
    - reaction_list (list): List of reactions.
    - xtb_external_path (str): Path to the XTB external script.
    - solvent (str): Solvent information.
    - reactive_complex_factor_list_intermolecular (list): List of reactive complex factors for intermolecular reactions.
    - reactive_complex_factor_list_intramolecular (list): List of reactive complex factors for intramolecular reactions.
    - freq_cut_off (int): Frequency cutoff.

    Returns:
    - list: List of successful reactions.
    """
    home_dir = os.getcwd()
    os.chdir(target_dir)
    ts_optimizer_list = []

    for rxn_idx, rxn_smiles in reaction_list:
        ts_optimizer_list.append(TSOptimizer(rxn_idx, rxn_smiles, xtb_external_path,
                                             solvent, None, reactive_complex_factor_list_intermolecular,
                                             reactive_complex_factor_list_intramolecular, freq_cut_off))

    print(f'{len(ts_optimizer_list)} reactions to process...')

    num_processes = multiprocessing.cpu_count()

    with concurrent.futures.ProcessPoolExecutor(max_workers=int(num_processes/2)) as executor:
        # Map the function to each object in parallel
        results = list(executor.map(obtain_individual_guesses, ts_optimizer_list))

    successful_reactions = [r for r in results if r is not None]

    print(f'Guesses have been generated for {len(successful_reactions)} out of {len(ts_optimizer_list)} reactions!')

In [ ]:
os.mkdir('work_dir/ts-tools')
target_dir = 'work_dir/ts-tools' 

successful_reactions = obtain_ts_guesses(target_dir, reaction_list, 
    xtb_external_path=None, solvent=None, reactive_complex_factor_list_intermolecular=None,
    reactive_complex_factor_list_intramolecular=None, freq_cut_off=150)

You can now inspect the preliminary TS guesses extracted from the reactive paths for every reaction in the 'work_dir/ts-tools'. Note that when Gaussian16 is available, a TS search with the Berny algorithm is started from each of the guesses iteratively (together with an IRC search), until a validated TS has been found. At xTB level of theory, final/validated TSs can typically be obtained on a minute scale, DFT level transition states can typically be obtained in a matter of several hours (depending on the basis set and functional selected).

## Exercises

1. Run TS-tools for the first 10 reactions of the list of [3+2] cycloaddition reactions generated earlier during this session (see auxiliary functions below to assist in the transformation of the reaction SMILES strings). Can you get an idea why some of the TS searches fail from inspection of the generated reactive paths?
2. Generate a small reaction space of Diels Alder reactions yourself based on the previous script and get TS guesses for each of them (aim for 20 reactions at most).

In [ ]:
os.chdir(home_dir)
os.getcwd()

In [ ]:
# auxiliary functions -- use them to add hydrogens to the reaction SMILES
from rdkit import Chem
import argparse
import sys
from collections import defaultdict


def parse_fragments(side_str):
    """Return list of RDKit Mol objects (with explicit Hs) for each '.'-separated fragment."""
    frags = [s for s in side_str.split('.') if s.strip()]
    mols = []
    for f in frags:
        m = Chem.MolFromSmiles(f, sanitize=True)
        if m is None:
            raise ValueError(f"Could not parse fragment SMILES: {f}")
        m = Chem.AddHs(m)
        mols.append(m)
    return mols


def highest_mapnum(mols):
    mx = 0
    for m in mols:
        for a in m.GetAtoms():
            if a.GetAtomMapNum() > mx:
                mx = a.GetAtomMapNum()
    return mx


def assign_h_maps_from_reactants(reactant_mols, start_map):
    """
    For each heavy atom (atom-map > 0) in the reactant_mols, collect its attached H atoms (sorted by
    atom index). Assign new unique atom-map numbers to unmapped Hs and return a dict
    heavy_map -> [h_map1, h_map2, ...] and the updated max map number.
    """
    mapping = defaultdict(list)  # heavy_mapnum -> list of h_mapnums (ordered)
    cur = start_map

    for m in reactant_mols:
        for atom in m.GetAtoms():
            heavy_map = atom.GetAtomMapNum()
            if atom.GetAtomicNum() == 1 or heavy_map == 0:
                continue

            # collect H neighbors in deterministic order
            hs = [nbr for nbr in atom.GetNeighbors() if nbr.GetAtomicNum() == 1]
            hs_sorted = sorted(hs, key=lambda x: x.GetIdx())

            for h in hs_sorted:
                hmap = h.GetAtomMapNum()
                if hmap == 0:
                    cur += 1
                    hmap = cur
                    h.SetAtomMapNum(hmap)
                mapping[heavy_map].append(hmap)

    return mapping, cur


def apply_reactant_mapping_to_products(product_mols, mapping, start_map):
    """
    For product heavy atoms with atom-map `am`, assign H atom-map numbers according to
    mapping[am] (ordered). If product has extra Hs, give them new unique numbers. Returns
    updated max map number and list of warnings (if counts differ).
    """
    cur = start_map
    warnings = []

    # Build a lookup for product heavy atoms by their atom-map number
    prod_heavy_lookup = defaultdict(list)  # heavy_map -> list of (mol, heavy_atom)
    for m in product_mols:
        for atom in m.GetAtoms():
            if atom.GetAtomicNum() == 1:
                continue
            am = atom.GetAtomMapNum()
            if am > 0:
                prod_heavy_lookup[am].append((m, atom))

    for heavy_map, desired_hmaps in mapping.items():
        # Find all heavy atoms in products with this mapping (normally one)
        prod_entries = prod_heavy_lookup.get(heavy_map, [])
        if not prod_entries:
            warnings.append(f"Heavy atom with map {heavy_map} appears in reactant but not in product.")
            continue

        # For each matching heavy atom in product (usually one), assign maps to its Hs
        for m, heavy_atom in prod_entries:
            prod_hs = [nbr for nbr in heavy_atom.GetNeighbors() if nbr.GetAtomicNum() == 1]
            prod_hs_sorted = sorted(prod_hs, key=lambda x: x.GetIdx())

            n_desired = len(desired_hmaps)
            n_prod = len(prod_hs_sorted)

            # assign up to the minimum count using the desired reactant-side ordering
            for i in range(min(n_desired, n_prod)):
                h = prod_hs_sorted[i]
                target_map = desired_hmaps[i]
                if h.GetAtomMapNum() != target_map:
                    h.SetAtomMapNum(target_map)

            # if product has more Hs than reactant mapping provided, give them fresh maps
            if n_prod > n_desired:
                for i in range(n_desired, n_prod):
                    h = prod_hs_sorted[i]
                    if h.GetAtomMapNum() == 0:
                        cur += 1
                        h.SetAtomMapNum(cur)

            # if product has fewer Hs than reactant, we cannot materialize missing atoms
            if n_prod < n_desired:
                warnings.append(
                    f"Different H-count for heavy atom map {heavy_map}: reactant has {n_desired}, product has {n_prod}. "
                    "Mapped first {min_count} Hs; remaining reactant H-map numbers have no product counterpart.".format(
                        min_count=min(n_desired, n_prod)
                    )
                )

    return cur, warnings


def mols_to_smiles(mols):
    return ".".join(Chem.MolToSmiles(m, canonical=False, allHsExplicit=True) for m in mols)


def update_reaction_h_mapping(rxn_smiles: str):
    if ">>" in rxn_smiles:
        react_str, prod_str = rxn_smiles.split(">>", 1)
    else:
        react_str = rxn_smiles
        prod_str = ""

    react_mols = parse_fragments(react_str)
    prod_mols = parse_fragments(prod_str) if prod_str.strip() else []

    # compute starting map number to avoid collisions
    start = max(highest_mapnum(react_mols), highest_mapnum(prod_mols))

    mapping, cur = assign_h_maps_from_reactants(react_mols, start)
    cur, warnings = apply_reactant_mapping_to_products(prod_mols, mapping, cur)

    react_out = mols_to_smiles(react_mols)
    prod_out = mols_to_smiles(prod_mols) if prod_mols else ""
    out = react_out + (" >> " + prod_out if prod_out else "")

    return out